In [1]:
#importing dependecies
#import praw  #not being used
import pandas as pd
#import config #not being used
from psaw import PushshiftAPI 
import datetime as dt

In [2]:
api = PushshiftAPI()

In [12]:
#adapted from https://psaw.readthedocs.io/en/latest/

gen = api.search_comments(q='GME', subreddit='wallstreetbets',filter=['body'],before=int(dt.datetime(2021, 1, 30,18,1).timestamp()))

max_response_cache = 10000
cache = []

for c in gen:
    cache.append(c[-1])

    # Omit this test to actually return all results. Wouldn't recommend it though: could take a while, but you do you.
    if len(cache) >= max_response_cache:
           break

C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [17]:
df = pd.DataFrame(cache)
df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
df

,body,created_utc,created
0,Ok eff beating around the bush. I got 1k to sa...,2021-01-31,1.612084e+09
1,I earn less than 1 usd per day! I bought 1 gme...,2021-01-31,1.612084e+09
2,Jesus.\n\nDo you ever walk out your front door...,2021-01-31,1.612084e+09
3,💎👐 🚀🚀🚀🚀🚀🚀🚀 only ppl to see GME fly by will be ...,2021-01-31,1.612084e+09
4,DFW here!! About 5 miles away from the $GME ho...,2021-01-31,1.612084e+09
...,...,...,...
9995,Thank you. What's your feeling about this move...,2021-01-30,1.612067e+09
9996,Is it still worth putting money into GME share...,2021-01-30,1.612067e+09
9997,If the tab is not picked up by the hedge funds...,2021-01-30,1.612067e+09
9998,"This is not a bad theory, BUT are you not miss...",2021-01-30,1.612067e+09


In [39]:
def ticker_reddit_scraper(ticker,year,month,day):
    #querie reddit for ticker
    gen = api.search_comments(q=ticker, 
                              subreddit='wallstreetbets',
                              after=int(dt.datetime(year, month, day).timestamp()),
                              before=int(dt.datetime(year, month, day+1).timestamp()),
                              filter=['body','author','subreddit']
                             )
    #max responses retrieved
    max_response=10000
    cache=[]
    for i in gen:
        #take only the dictionary from queries
        cache.append(i[-1])
        if len(cache) >= max_response_cache:
            break
    df = pd.DataFrame(cache)
    df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
    df = df.drop(columns=['created'])
    df.rename(columns={'author':'username','created_utc':'date'}, inplace=True)
    df=df[['date','body','username','subreddit']]
    return df


In [40]:
gme_yesterday = ticker_reddit_scraper('gme',2022,5,13)
gme_yesterday

,date,body,username,subreddit
0,2022-05-14,Let’s gooo GME !🚀🚀🚀,SufficientOccasion16,wallstreetbets
1,2022-05-14,GME![img](emote|t5_2th52|4270),i_spank_chickens,wallstreetbets
2,2022-05-14,Oh please these monkeys dont have money to sto...,Efficient-Library792,wallstreetbets
3,2022-05-14,Sooooo hedge with GME,Cowabunguss,wallstreetbets
4,2022-05-14,"I had GME at $40 and sold on the run, you know...",lJustLurkingl,wallstreetbets
...,...,...,...,...
452,2022-05-13,Investment journey so far. I have now held all...,saturnphive,wallstreetbets
453,2022-05-13,I have some GME still with a UK bank/broker. I...,Danboone003,wallstreetbets
454,2022-05-13,GME and hold through dividend stock split,Different_Cap_330,wallstreetbets
455,2022-05-13,Why can't it just pick itself up from its own ...,Philboyd_Studge,wallstreetbets


In [41]:
gme_yesterday.to_csv('test_gme_data.csv')